In [1]:
from data_prep import *
from charts import *
import pandas as pd

# Season Summaries

In [3]:

seasons = ["2021/22", "2022/23", "2023/24", "2024/25"]
for season in seasons:
    print(season)

    # Starts (by Position)
    plot_starts_by_position(1, season=season, fb_only=False).save(f"Charts/{season.replace('/','-')}/1s-squad-position.html")
    plot_starts_by_position(2, season=season, fb_only=False).save(f"Charts/{season.replace('/','-')}/2s-squad-position.html")
    # Appearances
    plot_games_by_player(0, min_games=10, season=season).save(f"Charts/{season.replace('/','-')}/appearances.html")
    plot_games_by_player(1, min_games=5, season=season).save(f"Charts/{season.replace('/','-')}/1s-appearances.html")
    plot_games_by_player(2, min_games=5, season=season).save(f"Charts/{season.replace('/','-')}/2s-appearances.html")
    # Points Scorers
    points_scorers_chart(1, season).save(f"Charts/{season.replace('/','-')}/1s-points.html")
    points_scorers_chart(2, season).save(f"Charts/{season.replace('/','-')}/2s-points.html")
    # Captains
    if season != ["2021/22", "2022/23"]:
        captains_chart(season).save(f"Charts/{season.replace('/','-')}/captains.html")
    # Cards
    card_chart(0, season).save(f"Charts/{season.replace('/','-')}/cards.html")

    # Lineouts
    lineout_chart(1, season).save(f"Charts/{season.replace('/','-')}/1s-lineouts.html")
    if season != "2021/22":
        lineout_chart(2, season).save(f"Charts/{season.replace('/','-')}/2s-lineouts.html")

2021/22
2022/23
2023/24
2024/25


# Trends


In [4]:
plot_games_by_player(0, min_games=10, season=None, agg=True).save("Charts/by-season/appearances.html")
captains_chart(season=None).save("Charts/by-season/captains.html")
card_chart(season=None).save("Charts/by-season/cards.html")
points_scorers_chart(squad=0, season=None).save("Charts/by-season/points.html")

# Set Piece

In [2]:
# Lineouts
for squad in [1, 2]:
    types = ["Numbers", "Area", "Hooker", "Jumper", "Setup", "Call", "Movement"]
    charts = []
    for i,t in enumerate(types):
        min = 10 if t in ["Hooker", "Jumper", "Call"] else 1
        chart = count_success_chart(t, squad, as_dict=True, min=min)
        filters = [{"filter": {"param": f"select{f}"}} for f in types if f != t]
        chart["transform"] = filters + chart["transform"]
        chart["title"]["fontSize"] = 36
        chart["title"]["color"] = "#202946"

        charts.append(alt.Chart.from_dict(chart))

    chart = (
        alt.vconcat(*charts)
        .resolve_scale(color="independent")
        .properties(
            title=alt.Title(
                text=f"{'1st' if squad==1 else '2nd'} XV Lineout Stats", 
                subtitle="Click on any bar to filter the other charts accordingly", 
                subtitleFontSize=14
            )
        )
        .save(f"Charts/by-season/{squad}s-lineouts.html")
    )

# Set Piece Head-to-Head
for squad in [1,2]:
    for event in ["scrum", "lineout"]:
        chart = set_piece_h2h_charts(squad, event)
        chart.save(f"Charts/by-season/{squad}s-{event}-h2h.html")

# 1st XV Game Analysis

In [6]:
pd.options.display.max_columns = None

analysis = sheet[0].batch_get(['B4:AZ'])[0]

df = pd.DataFrame(analysis, columns=analysis.pop(0))
df

id_cols = ["Date", "Opposition", "Home/Away", "PF", "PA"]
poss_terr = df[id_cols + ["BIP Time", "Poss (%)", "Terr (%)", "Own 22m (%)", "22m - Half (%)", "Half - 22m (%)", "Opp 22m (%)"]]
poss_terr


,Date,Opposition,Home/Away,PF,PA,BIP Time,Poss (%),Terr (%),Own 22m (%),22m - Half (%),Half - 22m (%),Opp 22m (%)
0,7 Sep 2024,London Cornish,H,16,18,36:05,37%,59%,9%,33%,45%,14%
1,12 Oct 2024,Hove,H,21,34,39:25,52%,56%,17%,27%,32%,24%
2,19 Oct 2024,Twickenham,A,33,41,34:23,49%,57%,11%,31%,30%,27%
3,26 Oct 2024,KCS Old Boys,H,29,33,38:31,67%,46%,23%,31%,34%,13%
4,9 Nov 2024,Eastbourne,A,36,33,34:35,52%,55%,17%,29%,39%,16%
5,16 Nov 2024,Trinity,H,14,48,35:15,57%,43%,25%,32%,31%,13%
6,15 Dec 2024,Old Haileyburians,H,18,5,39:30,53%,61%,16%,22%,39%,23%
7,4 Jan 2025,Weybridge Vandals,A,14,20,39:35,44%,46%,16%,38%,32%,13%


# Team Sheets

In [ ]:
def team_sheet_chart(
        squad=1, 
        names=None, 
        captain=None, 
        vc=None, 
        opposition=None, 
        home=True, 
        competition="Counties 1 Sussex",
        season="2023/24"
    ):

    if names is None:
        df = team_sheets(squad=1) 

        # Last row as dict
        team = df.iloc[-1].to_dict()


        label = f'{"1st" if squad==1 else "2nd"} XV vs {team["Opposition"]}({team["Home/Away"]})'
        captain = team["Captain"]
        vc = team["VC"]
        season = team["Season"]
        competition = team["Competition"]

        # Keep keys that can be converted to integers
        team = {int(k): v for k, v in team.items() if k.isnumeric() and v}

        # Convert team to dataframe with Number and Player columns
        team = pd.DataFrame(team.items(), columns=["Number", "Player"])

    else:
        label = f'{"1st" if squad==1 else "2nd"} XV vs {opposition} ({"H" if home else "A"})'

        # Convert names to Player column of a dataframe with Number column (1-len(names))
        team = pd.DataFrame({"Player": names, "Number": range(1, len(names)+1)})

    coords = pd.DataFrame([
                {"n": 1, "x": 10, "y": 81},
                {"n": 2, "x": 25, "y": 81},
                {"n": 3, "x": 40, "y": 81},
                {"n": 4, "x": 18, "y": 69},
                {"n": 5, "x": 32, "y": 69},
                {"n": 6, "x": 6, "y": 61},
                {"n": 7, "x": 44, "y": 61},
                {"n": 8, "x": 25, "y": 56},
                {"n": 9, "x": 20, "y": 42},
                {"n": 10, "x": 38, "y": 36},
                {"n": 11, "x": 8, "y": 18},
                {"n": 12, "x": 56, "y": 30},
                {"n": 13, "x": 74, "y": 24},
                {"n": 14, "x": 92, "y": 18},
                {"n": 15, "x": 50, "y": 10},
                {"n": 16, "x": 80, "y": 82},
                {"n": 17, "x": 80, "y": 74},
                {"n": 18, "x": 80, "y": 66},
                {"n": 19, "x": 80, "y": 58},
                {"n": 20, "x": 80, "y": 50},
                {"n": 21, "x": 80, "y": 42},
                {"n": 22, "x": 80, "y": 34},
                {"n": 23, "x": 80, "y": 26},
            ])
    team = team.merge(coords, left_on="Number", right_on="n", how="inner").drop(columns="n")

    # Add captain (C) and vice captain (VC) else None
    team["Captain"] = team["Player"].apply(lambda x: "C" if x == captain else "VC" if x == vc else None)

    team["Player"] = team["Player"].str.split(" ")

    team.to_dict(orient="records")

    with open("team-sheet-lineup.json") as f:
        chart = json.load(f)
    chart["data"]["values"] = team.to_dict(orient="records")
    chart["title"]["text"] = label
    chart["title"]["subtitle"] = f"{season} - {competition}"

    n_replacements = len(team) - 15
    
    y = 126 + (n_replacements * 64)
    chart["layer"][0]["mark"]["y2"] = y
    # return chart
    return alt.Chart.from_dict(chart)

In [ ]:
names = [
    "Guy Collins",
    "Ben Tottman",
    "Josh Brimecombe",
    "Ollie Toogood",
    "Sam Lindsay-McCall",
    "Josh Weaver",
    "Ryland Thomas",
    "Dan Billin",
    "Chris Taylor",
    "Luke Maker",
    "Jake Radcliffe",
    "Chris May-Miller",
    "Ted Hardisty",
    "Ali Moffatt",
    "Noah Roberts",
    "Or Shay",
    "Tom Mooney",
    "Jack Billin",
]

team_sheet_chart(names=names, captain="Ryland", opposition="London Cornish", competition="Counties 1 Surrey/Sussex")

ValidationError: {'condition': {'test': "datum['C'] == true", 'value': '#26d'}, 'value': '#139'} is not one of ['black', 'silver', 'gray', 'white', 'maroon', 'red', 'purple', 'fuchsia', 'green', 'lime', 'olive', 'yellow', 'navy', 'blue', 'teal', 'aqua', 'orange', 'aliceblue', 'antiquewhite', 'aquamarine', 'azure', 'beige', 'bisque', 'blanchedalmond', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgreen', 'darkgrey', 'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'greenyellow', 'grey', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 'khaki

In [ ]:
df1 = players()
df1 = df1[["Number", "Player", "Position"]]
df1 = df1[df1["Number"]<=15]

df1 = df1.groupby("Player").agg({"Position": lambda x: list(set(x))})
df1["N"] = df1["Position"].apply(len)
df1[df1["N"]>1].sort_values("N", ascending=False)

,Position,N
Player,,
Joe Terry,"[Back Three, Fly Half, Centre]",3
Ben Hogan,"[Fly Half, Back Three, Scrum Half]",3
Sam Spriddell,"[Back Three, Scrum Half, Centre]",3
Dan Billin,"[Back Row, Second Row, Centre]",3
Dave Bridges,"[Back Row, Prop, Second Row]",3
James Mitchell,"[Back Three, Scrum Half, Centre]",3
Ali Moffatt,"[Back Three, Centre]",2
Ted Hardisty,"[Back Three, Centre]",2
Rhys Skinner,"[Fly Half, Centre]",2


In [4]:


import requests
from bs4 import BeautifulSoup
import json

# URL of the EGRFC team statistics page
URL = "https://www.egrfc.com/teams/142069/statistics"

# Fetch and parse the page
def fetch_player_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to fetch the webpage.")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    
    table = soup.find_all("div", {"class": "no-grid-hide"})[0]

    headers = [h.text for h in table.find_all("div", {"class": "league-table__header"})]
    players = [p.text for p in table.find_all("div", "sc-bwzfXH iWTrZm")]
    positions = [p.text for p in table.find_all("div", "sc-bwzfXH fGYXYx")]
    
    players = [{"Player": p, "Position": [pos]} for p, pos in zip(players, positions)]

    return players

# Save to JSON file
def save_to_json(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)
    print(f"Player data saved to {filename}")

if __name__ == "__main__":
    player_data = fetch_player_data(URL)
    if player_data:
        save_to_json(player_data, "players.json")


Player data saved to players.json
